In [1]:
import glob
import numpy as np
import cv2
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D

SEED = 2017

Using TensorFlow backend.


In [3]:
n_classes = 5
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data',
                                                target_size = (150,150),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 subset = "training")

validation_set = train_datagen.flow_from_directory('data',
                                            target_size = (150,150),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            subset = "validation")

#Cats: 12500, #Dogs: 12500


In [7]:
def create_model(stride=1, padding='same', img_size=100):
    # Define architecture
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.) - 0.5, input_shape=(img_size, img_size, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding=padding, strides=stride))
    model.add(Conv2D(32, (3, 3), activation='relu', padding=padding, strides=stride))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 3), activation='relu', padding=padding, strides=stride))
    model.add(Conv2D(64, (3, 3), activation='relu', padding=padding, strides=stride))
    
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
    
    return model

In [8]:
img_size = 100

models = []
for stride in [1, 2]:
    for padding in ['same', 'valid']:
        model = create_model(stride, padding, img_size)
        pars = model.count_params()
        models.append(dict({'setting': '{}_{}'.format(stride, padding), 
                'model': model,
                'parameters': pars
                }))

In [9]:
models[0]['model'].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 50, 64)        36928     
__________

In [10]:
callbacks = [EarlyStopping(monitor='val_categorical_accuracy', patience=5)]

In [11]:
batch_size = 512
n_epochs = 500

history = []
for i in range(len(models)):
    print(models[i])
    history.append(
        models[i]['model'].
        fit_generator(training_set, epochs=n_epochs, validation_data=validation_set, callbacks=callbacks))

{'setting': '1_same', 'model': <keras.models.Sequential object at 0x7fece9877048>, 'parameters': 10305697}
Epoch 1/500
39/39 [==============================] - 77s - loss: 1.0154 - binary_accuracy: 0.4992 - val_loss: 0.6921 - val_binary_accuracy: 0.2928
Epoch 2/500
39/39 [==============================] - 80s - loss: 0.6825 - binary_accuracy: 0.5667 - val_loss: 0.6753 - val_binary_accuracy: 0.5758
Epoch 3/500
39/39 [==============================] - 54s - loss: 0.6811 - binary_accuracy: 0.5708 - val_loss: 0.6833 - val_binary_accuracy: 0.5592
Epoch 4/500
39/39 [==============================] - 67s - loss: 0.6742 - binary_accuracy: 0.5886 - val_loss: 0.6649 - val_binary_accuracy: 0.6379
Epoch 5/500
39/39 [==============================] - 58s - loss: 0.6702 - binary_accuracy: 0.6047 - val_loss: 0.6657 - val_binary_accuracy: 0.6109
Epoch 6/500
39/39 [==============================] - 72s - loss: 0.6691 - binary_accuracy: 0.6087 - val_loss: 0.6550 - val_binary_accuracy: 0.6436
Epoch 7/500

39/39 [==============================] - 59s - loss: 0.4910 - binary_accuracy: 0.7694 - val_loss: 0.4872 - val_binary_accuracy: 0.7600
Epoch 9/500
39/39 [==============================] - 65s - loss: 0.4852 - binary_accuracy: 0.7686 - val_loss: 0.4876 - val_binary_accuracy: 0.7662
Epoch 10/500
39/39 [==============================] - 62s - loss: 0.4576 - binary_accuracy: 0.7894 - val_loss: 0.4871 - val_binary_accuracy: 0.7654
Epoch 11/500
39/39 [==============================] - 64s - loss: 0.4479 - binary_accuracy: 0.7964 - val_loss: 0.4693 - val_binary_accuracy: 0.7641
Epoch 12/500
39/39 [==============================] - 57s - loss: 0.4253 - binary_accuracy: 0.8116 - val_loss: 0.4288 - val_binary_accuracy: 0.8031
Epoch 13/500
39/39 [==============================] - 62s - loss: 0.4172 - binary_accuracy: 0.8109 - val_loss: 0.4396 - val_binary_accuracy: 0.8203
Epoch 14/500
39/39 [==============================] - 66s - loss: 0.3924 - binary_accuracy: 0.8294 - val_loss: 0.4540 - val_bi

In [ ]:
for i in range(len(models)):
    plt.plot(range(len(history[i].history['val_categorical_accuracy'])), history[i].history['val_categorical_accuracy'], label=models[i]['setting'])
    print('Max accuracy model {}: {}'.format(models[i]['setting'], max(history[i].history['val_categorical_accuracy'])))
plt.title('Validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [17]:
for i in range(len(models)):
    plt.plot(range(len(history[i].history['val_categorical_accuracy'])), history[i].history['val_categorical_accuracy'], label=models[i]['setting'])
    print('Max accuracy model {}: {} (#parameters: {})'.format(models[i]['setting'], max(history[i].history['val_categorical_accuracy']), models[i]['parameters']))

Max accuracy model 1_same: 0.8375 (#parameters: 10305697)
Max accuracy model 1_valid: 0.8466796875 (#parameters: 7995553)
Max accuracy model 2_same: 0.840625 (#parameters: 102561)
Max accuracy model 2_valid: 0.8298828125 (#parameters: 82081)
